# PyKX Query Welcome Database
This notebook demonstrates querying the welcomedb database using PyKX.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env_kdb_1 import *

HDB_CLUSTER_NAME="cluster_welcomedb"

In [2]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# Connect to Cluster

In [3]:
# Query the HDB
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# Table Schema

In [4]:
schema_pdf = hdb("meta(`example)").pd()

display(schema_pdf)

,t,f,a
c,,,
date,b'd',,
sym,b's',,p
time,b'p',,
number,b'j',,


# Query Table

In [5]:
# Dates, counts, and analytics
hdb_pdf = hdb("select counts:count i, avg_num: avg number, avg_sq_num: avg sq number by date from example").pd()

# Number of Rows
hdb_rows = hdb("count example").py()

display(hdb_pdf)

# Number of Rows
print(f"Rows: {hdb_rows:,}")

,counts,avg_num,avg_sq_num
date,,,
2023-04-14,1000000,499800.464405,3.331963e+11
2023-04-15,1000000,499865.008159,3.332692e+11
2023-04-16,1000000,499912.379127,3.332060e+11
2023-04-17,1000000,500078.393386,3.334004e+11
2023-04-18,1000000,500264.684412,3.336606e+11
2023-04-19,1000000,499849.962912,3.333436e+11
2023-04-20,1000000,500082.257829,3.334472e+11
2023-04-21,1000000,500169.492354,3.336027e+11
2023-04-22,1000000,499903.047577,3.331833e+11


Rows: 10,000,000


In [6]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-06-27 13:59:43.520313
